In [ ]:
import os,sys,inspect
import random
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
import ipywidgets as widgets
import ipython_blocking
import numpy as np
import time
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil
from functools import partial
import json

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
testname="Тест игл"
display(Markdown('### 💠 {}'.format(testname)))



In [ ]:
from connect_device_v2.find_device import FindDevice,parse_config
box = FindDevice()
display(box)

In [ ]:
button = box.select_device_continue_button

In [ ]:
%blockrun button

In [ ]:
button1 = box.select_config_continue_button

In [ ]:
%blockrun button1

In [ ]:
inst = box.selected_device

In [ ]:
inst.write("SYST:PRES")

In [ ]:
date=datetime.now()
date=date.strftime('%Y-%m-%d %H:%M:%S')
device = inst.query("*IDN?")

device = device.replace(' ','')
device = device.split(',')
device_info = {'Время запуска':date,
              'device':device[0],
              'model':device[1],
              'serial':device[2],
              'version':device[3]}
#inst2 = ng.instance()
#путь к файлу с сегментами
currentDir=%pwd
#resultPath= currentDir +'\\frequency_plan.seg'
portCount = int(inst.query("SERV:PORT:COUN?"))
print(device_info)
print(portCount)

In [ ]:
config =  box.config
device = config['DEVICE']
fmin = float(inst.query(f'SENS:FREQ:STAR?'))
fmax = float(inst.query(f'SENS:FREQ:STOP?'))
points = int(device['points'])
freq = float(device['filter_IF'])
deltaqampl = float(device['deltaqampl'])
test_time = float(device['test_time'])
segment_factor=float(device['segment_factor'])
athenuator = int(device['athenuator'])
correctors_enable = bool(int(device['correctors_enable']))

In [ ]:
# def check_error():
#     responce = inst.query('SYST:ERR?')    
#     if "No error" not in responce:        
#         print(responce)
#         #pass
#     else:        
#         pass
#         #print('succes')

In [ ]:
def setup_mesuare_parametrs(con):
    con.write(f"SENS:CORR:COLL:CKIT 3")
    check_error(con)
    for i in range(1,3): 
        #print("SENS:CORR:COLL:CKIT:ORD:SEL 2")
        #con.write("SENS:CORR:COLL:CKIT:ORD:SEL 2")
        #check_error()
        #print(f"SENS:CORR:COLL:CKIT:STAN{i}:DEL 0")
        con.write(f"SENS:CORR:COLL:CKIT:STAN{i}:DEL 0")
        check_error(con)
        con.write(f"SENS:CORR:COLL:CKIT:STAN{i}:LOSS 0")
        check_error(con)
        con.write(f"SENS:CORR:COLL:CKIT:STAN{i}:C0 0")
        check_error(con)
        con.write(f"SENS:CORR:COLL:CKIT:STAN{i}:C1 0")
        check_error(con)
        con.write(f"SENS:CORR:COLL:CKIT:STAN{i}:C2 0")
        check_error(con)
        con.write(f"SENS:CORR:COLL:CKIT:STAN{i}:C3 0")
        check_error(con)

In [ ]:
def chunks(lst, n):
    new_data=[]
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        #new_data.append(lst[i:i + n])
        yield lst[i:i + n]
    
def create_query_string(header,data):
    query_string=''
    for i in header:
        query_string+=f'{i},'   
    for i in data:
        for j in i:
            query_string+=f'{str(j)},'        
    return(query_string[:-1])
    
def edit_segment_table(con,factor):
    data=con.query(f'SENSe1:SEGMent:DATA?').split(',')
    data_header=data[:7]
    segment_count=int(data_header[6])
    delimiter=int(len(data[7:])/segment_count)
    new_data=list(chunks(data[7:],int(delimiter)))    
    for i in range(0,len(new_data)):
        new_data[i][2]=int(new_data[i][2])*factor
    query_string=create_query_string(data_header,new_data)
    con.write(f'SENSe1:SEGMent:DATA {query_string}')
    

In [ ]:
def kalibration_for_port_list(con,ports_list:list):
    
    setup_mesuare_parametrs(con)    
    #print('port list=',ports_list)
    #print('port list=',type(ports_list))
    con.write('TRIG:SOUR INTernal')
    con.write('SYST:CORR 0')
    
    con.write("SERVice:SWEep:FREQuency:FACTory")
    edit_segment_table(con,segment_factor)
    enable_ifbf_for_all_segment(inst)
#     con.write("SENS:CORR:COLL:CKIT 2")
#     check_error()
#     con.write("SENS:CORR:COLL:CKIT:ORD:SEL 1")
#     check_error()
    con.write(f"SENS:CORR:COLL:CKIT 3")
    con.write("DISP:WIND:TRAC:Y:PDIV 0.05")
    
    con.write("SENS:CORR:STAT ON")
   
    # time.sleep(2)
    #time.sleep(3) 
    for port in ports_list:        
        con.write(f"SENS1:CORR:COLL:OPEN {port}")
        check_error(inst)
        con.write(f"SENS1:CORR:COLL:METH:OPEN {port}") 
        check_error(inst)    
        con.write("SENS:CORR:COLL:SAVE")
             
    check_error(inst)
    
    con.write("SENSe:SWEep:TYPE LINear")
    
    
#    прерводим прибор в режим внешнего управления и останавливаем развёртку
    inst.write("TRIG:SOUR BUS")
    check_error(inst)
    inst.write("INIT:CONT OFF")
    check_error(inst)
    con.write('SENSe:CORRection:COLLect:CKIT:RESet')
    check_error(inst)

In [ ]:
#kalibration_for_port_list(inst,[j for j in range(1,portCount+1)])


In [ ]:
# global tempstart
# tempstart = round(float(inst.query("SYST:TEMP:SENSor?").split('/')[0]),2)

In [ ]:
# Загружаем начальныке параметры частоты, фильтры, точки
#отключаем все корректоры
if (not correctors_enable):    
    disable_all_correctors(inst)
inst.write(f"SERVice:RFCTL:POWer:STATe {athenuator}")
inst.write(f"SERVice:RFCTL:POWer:ATT 10")
inst.write(f"SERV:RFCTL:POW:DAC 6554")
inst.write(f"SENS:FREQ:STAR {fmin}")
check_error(inst)
inst.write(f"SENS:FREQ:Stop {fmax}")
check_error(inst)
inst.write(f"SENS:SWE:POIN {points}")
check_error(inst)
inst.write(f"SENS:BAND {freq}")
check_error(inst)


In [ ]:
# прерводим прибор в режим внешнего управления и останавливаем развёртку
inst.write("TRIG:SOUR BUS")
check_error(inst)
inst.write("INIT:CONT OFF")
check_error(inst)
#inst.write("DISP:WIND:TRAC:Y:PDIV 0.01")
check_error(inst)
print()
        

In [ ]:
def setup_spur_trace_parameters(con,start,stop,points,port,kek=0):
    #print(con,start,stop,points,port)
    #kalibration_for_port_list(con,[j for j in range(port,port+1)])
    
    con.write('TRIG:SOUR INTernal')
    con.write('INIT:CONT 1')
    con.write("SENSe:SWEep:TYPE LINear")
    con.write(f"SENS:FREQ:STAR {start}")  
    con.write(f"SENS:FREQ:Stop {stop}")   
    con.write(f"SENS:SWE:POIN {points}")    
    con.write(f"CALC:PAR1:DEF s{port}{port}")    
    con.write(f"DISP:WIND:TRAC{1}:Y:PDIV 0.01")
    

In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from itertools import product,chain

# def btn_click(_,con,fstart,fstop,points,port): 
#     setup_spur_trace_parameters(con,fstart,fstop,points,port)
            

def moving_average(x, w):        
        ext=np.pad(x, (int(w/2),0),'constant')        
        ext=np.pad(ext, (0,int(w/2)-1),'constant')        
        return np.convolve(ext, np.ones(w), 'valid') / w

def get_peaks_index(peaks,negative_peaks):
    result = []
    for i in peaks:
        result.append(i)
    for i in negative_peaks:
        result.append(i)
    return result
def find_spur_v2(port,fstart,fstop,pointizm):
    global spur_table 
    global spur_counter    
#    time.sleep(0.5)    
    data = inst.query(f"CALC1:TRAC1:DATA:FDAT?")
    check_error(inst)
    data = data.split(',')[::2] 
    frequency_data = inst.query("SENS:FREQ:DATA?").split(',')
    check_error(inst)
    frequency_data = np.array(list(map(float,frequency_data)))
    data = list(map(float, data))          
    x= np.array(data)
    ampl=deltaqampl
    arr_averg=moving_average(x,20)
    arr1=arr_averg+ampl
    arr2=arr_averg-ampl
    peaks, _ = find_peaks(x, height=arr1,prominence=ampl)
    negative_peaks, _ = find_peaks(-x, height=-arr2, prominence=ampl)
    if (len(peaks)!=0 or len(negative_peaks)!=0):
       
        print(f's{port}{port}')
        print(f'start: {fstart}, stop: {fstop}, points: {pointizm}, temp: {temperature}')         
        button1 = widgets.Button(description = 'Установить параметры в приложении')
        click=partial(setup_spur_trace_parameters,inst,fstart,fstop,pointizm,port)        
        button1.on_click(click)
        display(button1)
        
        spur_info = {'id':f'{spur_counter}',
                     'trace':f's{port}{port}',
                     'start':f'{fstart}',
                     'stop':f'{fstop}',
                     'points':f'{pointizm}',
                     'spur_frequency':[frequency_data[i] for i in get_peaks_index(peaks,negative_peaks)],
                     'temp':temperature}
        spur_table['spurs'].append(spur_info)
        
        file_name=str(device_info['model'])
        with open(f'{file_name}.json') as json_file:
            data = json.load(json_file)
            data['spurs'].append(spur_info)
        with open(f'{file_name}.json',"w") as json_file:
            json.dump(data, json_file,indent = 4)
        spur_counter+=1
        plt.plot(frequency_data,x)
        plt.plot(frequency_data,arr1, color="gray")
        plt.plot(frequency_data,arr2, color="gray")
        plt.plot(frequency_data[peaks], x[peaks], "x")
        plt.plot(frequency_data[negative_peaks], x[negative_peaks], "x")
        plt.plot(frequency_data,np.zeros_like(x), "--", color="gray")
        plt.show()
        print('-------------------------------------')
    
    

In [ ]:
def test_round():        
    #print('### testround')
    global temperature
    current_temp = float(inst.query("SYST:TEMP:SENSor?").split('/')[0])
    if (float(current_temp)-float(temperature)>=0.5):
        print('calibrate')
        kalibration_for_port_list(inst,[j for j in range(1,portCount+1)]) 
        temperature=current_temp
    
    global tempstart
    pointizm = points + int(2000 * random.random())
    fstart = fmin + ((fmax - fmin) * random.random())
    fstop = fmax - ((fmax - fstart) * random.random())   
    start = time.time()    
    inst.write(f"SENS:FREQ:STAR {fstart}")  
    inst.write(f"SENS:FREQ:Stop {fstop}")   
    inst.write(f"SENS:SWE:POIN {pointizm}")    
    inst.write(f"SENS:BAND {freq}")    
    # !!!!!!! фиксированная мощьность. 0 дБм
    inst.write("SOUR:POW 0")
    inst.write(f"DISP:WIND:TRAC{1}:Y:PDIV 0.01")
      
    for i in range(1,portCount+1):  
        inst.write(f"CALC:PAR1:DEF s{i}{i}")
        check_error(inst)
        #check_error()
        #inst.write(f"SENSe1:SWEep:TYPE LINear")
        #check_error()
        #print('INIT')
        print('начало росчерка')
        t= time.time()
        inst.write("INIT")
        check_error(inst)
        #print("TRIG:SING")
        inst.write("TRIG:SING")         
        check_error(inst)
        inst.query("*OPC?")
        
        print(f'конец росчерка, время {time.time()-t}')
        inst.query("SYST:ERR?")
        spur_find_time= time.time()
        find_spur_v2(i,fstart,fstop,pointizm)  
        print(f"время на поиск иглы {time.time()-spur_find_time}")
        #print("init")
    
    
        

In [ ]:
inst.timeout=30000

In [ ]:
import time
temperature = float(inst.query("SYST:TEMP:SENSor?").split('/')[0])

spur_counter = 0
spur_table  = {'spurs':[]}
file_name=str(device_info['model'])
with open(f'{file_name}.json', 'w') as json_file:
    json.dump(spur_table, json_file,indent = 4)
#test_time = 5 # min
result = []
start= time.time()
pbar = tqdm_notebook(total=100)
old_value=0
oldprogress=0
iter_count = 0
inst.write(f"SENSe1:SWEep:TYPE LINear")
kalibration_for_port_list(inst,[j for j in range(1,portCount+1)]) 
while ((time.time()-start)<test_time*60):
#for i in range(1):
    #print(temperature)
    test_round()
    progress=(time.time()-start)    
    old_value=progress*100/(test_time*60)    
    pbar.update(old_value-oldprogress)
    oldprogress = old_value
    iter_count+=1
pbar.close()
    

In [ ]:
def convert_frequenscy_string(string):
    if (type(string)is str):
        string=string.split('.')[0]
        result='{0:,}'.format(int(string)).replace(',', ' ')
        return result
    if (type(string)is not str):
        for i in range(len(string)):
            #string[i]=string[i].split('.')[0]
            result='{0:,}'.format(int(string[i])).replace(',', ' ')
            return result
            

import pandas as pd
from IPython.display import display, HTML
from ipywidgets import Button, HBox, VBox,widgets
import ipywidgets
# load data using Python JSON module
with open(f'{file_name}.json','r') as f:
    data = json.loads(f.read())
for spur in data['spurs']:
    spur['start']=convert_frequenscy_string(spur['start'])
    spur['stop']=convert_frequenscy_string(spur['stop'])
    spur['spur_frequency']=convert_frequenscy_string(spur['spur_frequency'])
#df_nested_list = pd.json_normalize(data, record_path =['spurs'])
df = pd.DataFrame.from_dict(data['spurs'])
df = df[['id','trace','start','stop','points','spur_frequency','temp']]
df.set_index('id', inplace=True)
button = widgets.Button(description='click')
def obc(b):
    click_list.append((pd.to_datetime('now'),1)) 
button.on_click(obc)
table=HTML(df.style.set_table_attributes('class="table" border="1"').render())
display(table)

In [ ]:
file_data = table.data
my_button = SaveFilesButton(file_data)
my_button # This will display the button in the context of Jupyter Notebook